# Vogel's approximation
## For assignment1 (use void demand)

In [1]:
import numpy as np
import pandas as pd

# store the names of plant locations
names=['Mexico','Canada','Chile','Frankfurt','Austin','Japan']

# demand is already known
d=np.array([3,2.6,16,20,26.4,11.9])

# capacity is already known
s=np.array([22,3.7,4.5,47,18.5,5])

# since sum(s) is larger than sum(d), can create a void demander who will import all remaining products
d=np.array([3,2.6,16,20,26.4,11.9,sum(s)-sum(d)])

# create cost matrix
# production cost
c_production=np.array([[92.63,93.25,112.31,73.34,89.15,149.24]])
c_production=np.repeat(c_production, len(d)).reshape(len(s),len(d))

# duties cost, use 0 for the void demander
c_duties=np.array([[0.6,0,0.5,0.095,0.045,0.06,0]])
c_duties=np.repeat(c_duties, len(s)).reshape(len(s),len(d),order="F")
c_duties[range(len(s)),range(len(s))]=0 # set diagnol as 0
c_duties=c_duties+1

# shipping cost, use 0 for the void demander
c_shipping=np.array([[0,11.4,7,11,11,14,0],
                     [11,0,9,11.5,6,13,0],
                     [7,10,0,13,10.4,14.3,0],
                     [10,11.5,12.5,0,11.2,13.3,0],
                     [10,6,11,10,0,12.5,0],
                     [14,13,12.5,14.2,13,0,0]])

c=c_production*c_duties+c_shipping

# change cost of the void d to 0, so that the function to be optimized remains unchanged
c[:,-1]=0

In [2]:
# define VAM optimization function
def vam_opt(s,d,c,show_iter=True):
    '''
    s, the original supply vector
    d, the original demand vector
    c, the original cost matrix
    show_iter, if not 0, print the results after each iteration
    '''
    
    # declare local variables
    s0,d0,c0=s.copy(),d.copy(),c.copy()
    
    # declare the matrix to be optimized
    x0=np.zeros((len(s0), len(d0)))

    # use a large m to represent an infinite large
    m=c0.max()*1000
    
    # start optimization
    num_iter=0
    while True:
        # exclude optimized rows and columns
        a=np.repeat(s0!=0, len(d0)).reshape(c0.shape)
        b=np.repeat(d0!=0, len(s0)).reshape(c0.shape,order="F")
        c0[(a & b)!=True]=m

        # calculate opportunity cost arrays
        minc_s=np.repeat(c0.min(axis=1), c0.shape[1]).reshape(c0.shape)
        opcost_s=c0-minc_s
        opcost_s=np.sort(opcost_s, axis=1)
        opcost_s=opcost_s[:,1]
        # opcost_s[opcost_s>(0.5*m)]=0 # not necessary

        minc_d=np.repeat(c0.min(axis=0), c0.shape[0]).reshape(c0.shape,order="F")
        opcost_d=c0-minc_d
        opcost_d=np.sort(opcost_d, axis=0)
        opcost_d=opcost_d[1,:]
        # opcost_d[opcost_d>(0.5*m)]=0 # not necessary

        # find the index of s and d to be optimized
        maxcost=max(max(opcost_s),max(opcost_d))
        rows=opcost_s==maxcost
        cols=opcost_d==maxcost

        index1=np.repeat(rows, c0.shape[1]).reshape(c0.shape)
        index2=np.repeat(cols, c0.shape[0]).reshape(c0.shape,order="F")
        ctmp=c0.copy()
        ctmp[(index1 | index2)!=True]=m

        i=np.where(ctmp==ctmp.min())[0][0]
        j=np.where(ctmp==ctmp.min())[1][0]

        # update x,s,d
        if min(s0[i],d0[j])==0:
            break
        x0[i,j]=min(s0[i],d0[j])
        s0[i]=s0[i]-x0[i,j]
        d0[j]=d0[j]-x0[i,j]
        
        num_iter+=1

        if show_iter:
            print('After %d iteration(s):' % num_iter)
            print('x%d=\n' % num_iter,x0)
            print('d%d=\n' % num_iter,d0)
            print('s%d=\n' % num_iter,s0)
            print()
    return x0

In [3]:
# define output farmatting function
def output_table(x):
    # x, the optimized matrix
    x_df=pd.DataFrame(x[:,:len(names)])
    x_df.columns,x_df.index=names,names
    x_df['Production Plan']=x_df.sum(axis=1)
    x_df['Capacity(%)']=(x_df['Production Plan']/s*100).round(2).astype(str)+'%'
    display(x_df)

### Q2 use all capacity

In [4]:
s_q2=s.copy()
d2=np.array([3,2.6,16,20,26.4,11.9,sum(s_q2)-sum(d[:-1])])

In [5]:
x_q2=vam_opt(s=s_q2,d=d2,c=c,show_iter=True)
print('Total cost: %.4f' % sum(sum(x_q2*c)/100))

After 1 iteration(s):
x1=
 [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 5.]]
d1=
 [ 3.   2.6 16.  20.  26.4 11.9 15.8]
s1=
 [22.   3.7  4.5 47.  18.5  0. ]

After 2 iteration(s):
x2=
 [[0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  4.5]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  5. ]]
d2=
 [ 3.   2.6 16.  20.  26.4 11.9 11.3]
s2=
 [22.   3.7  0.  47.  18.5  0. ]

After 3 iteration(s):
x3=
 [[0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  3.7]
 [0.  0.  0.  0.  0.  0.  4.5]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  5. ]]
d3=
 [ 3.   2.6 16.  20.  26.4 11.9  7.6]
s3=
 [22.   0.   0.  47.  18.5  0. ]

After 4 iteration(s):
x4=
 [[0.  0.  0.  0.  0.  0.  7.6]
 [0.  0.  0.  0.  0.  0.  3.7]
 [0.  0.  0.  0.  0.  0.  4.5]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0. 

In [6]:
output_table(x_q2)

,Mexico,Canada,Chile,Frankfurt,Austin,Japan,Production Plan,Capacity(%)
Mexico,3.0,2.6,0.0,0.0,7.9,0.9,14.4,65.45%
Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%
Chile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%
Frankfurt,0.0,0.0,16.0,20.0,0.0,11.0,47.0,100.0%
Austin,0.0,0.0,0.0,0.0,18.5,0.0,18.5,100.0%
Japan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%


### Q3 use 85% of all capacity

In [7]:
s_q3=s*0.85      
d3=np.array([3,2.6,16,20,26.4,11.9,sum(s_q3)-sum(d[:-1])])

In [8]:
x_q3=vam_opt(s=s_q3,d=d3,c=c,show_iter=True)
print('Total cost: %.4f' % sum(sum(x_q3*c)/100))

After 1 iteration(s):
x1=
 [[0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   4.25]]
d1=
 [ 3.     2.6   16.    20.    26.4   11.9    1.445]
s1=
 [18.7    3.145  3.825 39.95  15.725  0.   ]

After 2 iteration(s):
x2=
 [[0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    1.445]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    4.25 ]]
d2=
 [ 3.   2.6 16.  20.  26.4 11.9  0. ]
s2=
 [18.7    3.145  2.38  39.95  15.725  0.   ]

After 3 iteration(s):
x3=
 [[3.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    1.445]
 [0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0. 

In [9]:
output_table(x_q3)

,Mexico,Canada,Chile,Frankfurt,Austin,Japan,Production Plan,Capacity(%)
Mexico,3.0,0.0,5.57,0.0,10.130,0.0,18.700,85.0%
Canada,0.0,2.6,0.00,0.0,0.545,0.0,3.145,85.0%
Chile,0.0,0.0,2.38,0.0,0.000,0.0,2.380,52.89%
Frankfurt,0.0,0.0,8.05,20.0,0.000,11.9,39.950,85.0%
Austin,0.0,0.0,0.00,0.0,15.725,0.0,15.725,85.0%
Japan,0.0,0.0,0.00,0.0,0.000,0.0,0.000,0.0%


### Q4 not using Chile, use 90% of all capacity

In [10]:
s_q4=s*0.9
s_q4[[i=='Chile' for i in  names]]=0
d4=np.array([3,2.6,16,20,26.4,11.9,sum(s_q4)-sum(d[:-1])])

In [11]:
x_q4=vam_opt(s=s_q4,d=d4,c=c,show_iter=True)
print('Total cost: %.4f' % sum(sum(x_q4*c)/100))

After 1 iteration(s):
x1=
 [[0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  4.5]]
d1=
 [ 3.    2.6  16.   20.   26.4  11.9   2.18]
s1=
 [19.8   3.33  0.   42.3  16.65  0.  ]

After 2 iteration(s):
x2=
 [[0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   2.18]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   4.5 ]]
d2=
 [ 3.   2.6 16.  20.  26.4 11.9  0. ]
s2=
 [19.8   1.15  0.   42.3  16.65  0.  ]

After 3 iteration(s):
x3=
 [[3.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   2.18]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   4.5 ]]
d3=
 [ 0.   2.6 16.  20.  26.4 11.9  0. ]
s3=
 [16.8   1.15  0.   42.3  16.65  0.  ]

After 4 i

In [12]:
output_table(x_q4)

,Mexico,Canada,Chile,Frankfurt,Austin,Japan,Production Plan,Capacity(%)
Mexico,3.0,1.45,0.0,0.0,9.75,5.6,19.80,90.0%
Canada,0.0,1.15,0.0,0.0,0.00,0.0,1.15,31.08%
Chile,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0%
Frankfurt,0.0,0.00,16.0,20.0,0.00,6.3,42.30,90.0%
Austin,0.0,0.00,0.0,0.0,16.65,0.0,16.65,90.0%
Japan,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0%
